#  Notebook 2 : Train Models

CropHealth Detection - Entraînement des 5 modèles

In [ ]:
📌 Cellule 1 : Vérifier environnement

In [ ]:
import torch
from pathlib import Path

print("="*70)
print("🔍 VÉRIFICATION ENVIRONNEMENT")
print("="*70)

# Check GPU
assert torch.cuda.is_available(), "❌ GPU non disponible ! Retourner au Notebook 1"
print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

# Check project
assert Path('train.py').exists(), "❌ Projet non cloné ! Retourner au Notebook 1"
print(f"✅ Projet: CropHealth_Detection_PFE")

# Check datasets
assert Path('data/yolo_crop').exists(), "❌ Dataset YOLO non trouvé ! Retourner au Notebook 1"
print(f"✅ Dataset YOLO: {len(list(Path('data/yolo_crop/train/images').glob('*.jpg')))} images")

assert Path('data/coco_crop').exists(), "❌ Dataset COCO non trouvé ! Retourner au Notebook 1"
print(f"✅ Dataset COCO: OK")

print("\n✅ Environnement prêt pour le training !")
print("="*70)

Cellule 2 : Configuration du training

In [1]:
# Configuration globale
TRAIN_CONFIG = {
    'device': 'cuda',
    'monitor_tensorboard': True,  # Lancer TensorBoard pendant training
    'save_interval': 5,  # Sauvegarder checkpoints tous les N epochs
}

# Modèles à entraîner (mettre False pour skip)
MODELS_TO_TRAIN = {
    'ssd': True,
    'yolov8n': True,
    'efficientdet': True,
    'fasterrcnn': True,
    'fasterrcnn_light': True,
}

print("="*70)
print("⚙️  CONFIGURATION DU TRAINING")
print("="*70)
print(f"\nDevice: {TRAIN_CONFIG['device']}")
print(f"TensorBoard: {TRAIN_CONFIG['monitor_tensorboard']}")
print(f"\nModèles à entraîner:")
for model, enabled in MODELS_TO_TRAIN.items():
    status = "✅" if enabled else "⏭️ "
    print(f"  {status} {model}")
print("="*70)

⚙️  CONFIGURATION DU TRAINING

Device: cuda
TensorBoard: True

Modèles à entraîner:
  ✅ ssd
  ✅ yolov8n
  ✅ efficientdet
  ✅ fasterrcnn
  ✅ fasterrcnn_light


📌 Cellule 3 : Lancer TensorBoard (optionnel)

In [ ]:
if TRAIN_CONFIG['monitor_tensorboard']:
    print("\n📊 Lancement de TensorBoard...")

    %load_ext tensorboard
    %tensorboard --logdir runs/ --port 6006

    print("✅ TensorBoard lancé sur le port 6006")
    print("💡 Cliquer sur le lien ci-dessus pour visualiser")
else:
    print("⏭️  TensorBoard désactivé")

📌 Cellule 4 : Training SSD MobileNetV3

In [ ]:
import time

if MODELS_TO_TRAIN['ssd']:
    print("\n" + "="*70)
    print("🚀 TRAINING SSD MobileNetV3")
    print("="*70 + "\n")

    start_time = time.time()

    !python train.py \
        --model ssd \
        --data data/yolo_crop \
        --device {TRAIN_CONFIG['device']}

    elapsed = (time.time() - start_time) / 60
    print(f"\n✅ SSD training terminé en {elapsed:.1f} minutes")
else:
    print("⏭️  SSD training skipped")

📌 Cellule 5 : Training YOLOv8n

In [ ]:
if MODELS_TO_TRAIN['yolov8n']:
    print("\n" + "="*70)
    print("🚀 TRAINING YOLOv8n")
    print("="*70 + "\n")

    start_time = time.time()

    !python train_yolo.py \
        --data data/yolo_crop/data.yaml \
        --device 0 \
        --cache

    elapsed = (time.time() - start_time) / 60
    print(f"\n✅ YOLOv8n training terminé en {elapsed:.1f} minutes")
else:
    print("⏭️  YOLOv8n training skipped")

📌 Cellule 6 : Training EfficientDet-D0

In [ ]:
if MODELS_TO_TRAIN['efficientdet']:
    print("\n" + "="*70)
    print("🚀 TRAINING EfficientDet-D0")
    print("="*70 + "\n")

    start_time = time.time()

    !python train.py \
        --model efficientdet \
        --data data/coco_crop \
        --device {TRAIN_CONFIG['device']}

    elapsed = (time.time() - start_time) / 60
    print(f"\n✅ EfficientDet training terminé en {elapsed:.1f} minutes")
else:
    print("⏭️  EfficientDet training skipped")

# 📌 Cellule 7 : Training Faster R-CNN ResNet50

In [ ]:
if MODELS_TO_TRAIN['fasterrcnn']:
    print("\n" + "="*70)
    print("🚀 TRAINING Faster R-CNN ResNet50")
    print("="*70 + "\n")

    start_time = time.time()

    !python train.py \
        --model fasterrcnn \
        --data data/yolo_crop \
        --device {TRAIN_CONFIG['device']}

    elapsed = (time.time() - start_time) / 60
    print(f"\n✅ Faster R-CNN training terminé en {elapsed:.1f} minutes")
else:
    print("⏭️  Faster R-CNN training skipped")

📌 Cellule 8 : Training Faster R-CNN Light

In [ ]:
if MODELS_TO_TRAIN['fasterrcnn_light']:
    print("\n" + "="*70)
    print("🚀 TRAINING Faster R-CNN Light (MobileNetV3)")
    print("="*70 + "\n")

    start_time = time.time()

    !python train.py \
        --model fasterrcnn_light \
        --data data/yolo_crop \
        --device {TRAIN_CONFIG['device']}

    elapsed = (time.time() - start_time) / 60
    print(f"\n✅ Faster R-CNN Light training terminé en {elapsed:.1f} minutes")
else:
    print("⏭️  Faster R-CNN Light training skipped")

📌 Cellule 9 : Résumé des trainings

In [ ]:
import glob
import pandas as pd
from pathlib import Path

print("\n" + "="*70)
print("📊 RÉSUMÉ DES TRAININGS")
print("="*70 + "\n")

# Chercher tous les checkpoints
checkpoints_info = []

models_patterns = {
    'SSD': 'runs/CropHealth_SSD_*/best.pt',
    'YOLOv8n': 'runs/CropHealth_YOLOv8n_*/weights/best.pt',
    'EfficientDet': 'runs/CropHealth_EfficientDet_*/best.pt',
    'Faster R-CNN': 'runs/CropHealth_FasterRCNN_*/best.pt',
    'Faster R-CNN Light': 'runs/CropHealth_FasterRCNN_light_*/best.pt',
}

for model_name, pattern in models_patterns.items():
    checkpoints = sorted(glob.glob(pattern))

    if checkpoints:
        latest = checkpoints[-1]
        size_mb = Path(latest).stat().st_size / (1024**2)
        run_dir = Path(latest).parent.parent if 'yolo' in pattern.lower() else Path(latest).parent

        checkpoints_info.append({
            'Modèle': model_name,
            'Checkpoint': latest,
            'Taille (MB)': f"{size_mb:.1f}",
            'Run': run_dir.name,
            'Status': '✅'
        })
    else:
        checkpoints_info.append({
            'Modèle': model_name,
            'Checkpoint': 'N/A',
            'Taille (MB)': 'N/A',
            'Run': 'N/A',
            'Status': '❌'
        })

# Afficher tableau
df = pd.DataFrame(checkpoints_info)
print(df.to_string(index=False))

print("\n" + "="*70)

# Compter succès
success_count = sum(1 for info in checkpoints_info if info['Status'] == '✅')
print(f"\n✅ {success_count}/{len(models_patterns)} modèles entraînés avec succès !")

if success_count < len(models_patterns):
    print("\n⚠️  Certains modèles n'ont pas été entraînés.")
    print("💡 Vérifier les logs ci-dessus pour les erreurs.")

📌 Cellule 10 : Visualiser courbes TensorBoard

In [ ]:
from IPython.display import Image, display

print("\n" + "="*70)
print("📈 COURBES D'ENTRAÎNEMENT")
print("="*70 + "\n")

# Chercher fichiers TensorBoard
tb_dirs = list(Path('runs').glob('CropHealth_*'))

if tb_dirs:
    print(f"✅ {len(tb_dirs)} runs trouvés")
    print("\n💡 Visualisation dans TensorBoard ci-dessus")
    print("💡 Ou exécuter: %tensorboard --logdir runs/")
else:
    print("❌ Aucun run trouvé")

📌 Cellule 11 : Sauvegarder checkpoints vers Drive

In [ ]:
import shutil
from datetime import datetime

print("\n" + "="*70)
print("💾 SAUVEGARDE VERS GOOGLE DRIVE")
print("="*70 + "\n")

# Créer dossier backup avec timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M')
backup_dir = f'/content/drive/MyDrive/CropHealth_Checkpoints_{timestamp}'

print(f"📁 Création du backup: {backup_dir}")
Path(backup_dir).mkdir(parents=True, exist_ok=True)

# Copier runs
if Path('runs').exists():
    print("📦 Copie des runs...")
    shutil.copytree('runs', f'{backup_dir}/runs', dirs_exist_ok=True)
    print("✅ Runs copiés")

print(f"\n✅ Backup terminé !")
print(f"📁 Location: {backup_dir}")

📌 Cellule 12 : Résumé final

In [ ]:
import json

print("\n" + "="*70)
print("🎉 TRAINING COMPLET !")
print("="*70 + "\n")

training_summary = {
    "Modèles entraînés": success_count,
    "Checkpoints": str(Path('runs').absolute()),
    "Backup Drive": backup_dir,
    "Durée totale": "Voir logs ci-dessus",
    "Prochaine étape": "Notebook 3 : Évaluation & Prédictions"
}

print(json.dumps(training_summary, indent=2, ensure_ascii=False))

print("\n" + "="*70)
print("➡️  Passez au Notebook 3 : Évaluation")
print("="*70)